### Test database

Let's figure out how the database works and what sort of data and analysis is availabe 

documentation: https://pypi.org/project/pybliometrics/
and: https://pybliometrics.readthedocs.io/en/stable/

In [1]:
pip install pybliometrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [9]:
from pybliometrics.scopus.utils import config
print(config['Authentication']['APIKey'])  # Show keys
print(config)

ac5f4d2994dbec80ddc553e23e50f88b


In [6]:
# Document-specific information
from pybliometrics.scopus import AbstractRetrieval
ab = AbstractRetrieval("10.1016/j.softx.2019.100263")
ab.title

Scopus401Error: The requestor is not authorized to access the requested view or fields of the resource

In [ ]:

'pybliometrics: Scriptable bibliometrics using a Python interface to Scopus'
>>> ab.publicationName
'SoftwareX'
>>> ab.authors
[Author(auid=57209617104, indexed_name='Rose M.E.', surname='Rose',
 given_name='Michael E.', affiliation='60105007'),
 Author(auid=7004212771, indexed_name='Kitchin J.R.', surname='Kitchin',
 given_name='John R.', affiliation='60027950')]
>>>
>>> # Author-specific information
>>> from pybliometrics.scopus import AuthorRetrieval
>>> au2 = AuthorRetrieval(ab.authors[1].auid)
>>> au2.h_index
34
>>> au1 = AuthorRetrieval(ab.authors[0].auid)
>>> au1.affiliation_current
[Affiliation(id=60105007, parent=None, type='parent', relationship='author',
 afdispname=None, preferred_name='Max Planck Institute for Innovation and Competition',
 parent_preferred_name=None, country_code='deu', country='Germany',
 address_part='Marstallplatz 1', city='Munich', state='Bayern',
 postal_code='80539', org_domain='ip.mpg.de', org_URL='http://www.ip.mpg.de/')]
>>>
>>> # Affiliation information
>>> from pybliometrics.scopus import AffiliationRetrieval
>>> aff1 = AffiliationRetrieval(au1.affiliation_current[0].id)
>>> aff1.author_count